# Blockchain and criptocurrency implementation in Python

## Packages necesarios

In [74]:
from datetime import datetime as dt # module to get timestamp
from hashlib import sha256 as sha # object to compute the sha256 digest of an string

***

## Transacciones

Se crea un objeto *Transaction* que registrará el **origen y destinatario de una transacción**, el **importe total** de la transacción y el momento en el que se produce la transacción

In [75]:
class Transaction(object):
    """
    Each transaction between two parties
    
    :param origin: who makes the transaction
    :param destination: to whom the transaction is made
    :param ammount: how many coins are transfered in the transaction
    """
    def __init__(self, origin, destination, ammount, timestamp):
        self.origin = origin
        self.destination = destination
        self.ammount = ammount
        self.timestamp = timestamp
        
    def __str__(self):
        ret = f"- " + f"Transaction of {self.ammount} coins from {self.origin} to {self.destination}."
        ret += f" " + f"Transaciton timestamp: {self.timestamp}"
        return ret
       

La representación de una transacción es la siguiente:

In [76]:
print(Transaction("Alice", "Bob", 20, dt.now()))

- Transaction of 20 coins from Alice to Bob. Transaciton timestamp: 2018-11-09 13:46:58.944707


***

## Bloques

Un bloque tiene los siguientes atributos:
-  **Transacciones**: Listado de todas las transacciones que forman parte del bloque. En este caso, será una lista de objetos *Transaction*

In [77]:
class Block(object):
    """
    Each block in the blockchain. A block is a list of transactions
    """
    
    def __init__(self):
        self.id = 1
        self.timestamp = None
        self.transactions = []
        self.hash_previous_block = ''
        
    def __add__(self, other):
        """
        Add a transaction to the block
        """
        if isinstance(other, Transaction):
            self.transactions.append(other)
        return self
        
    def __radd__(self, other):
        return self.__add__(self, other)
    
    def __sub__(self, other):
        """
        Delete a transaction from the block
        """
        if isinstance(other, Transaction) and other in self.transactions:
            self.transactions = [x for x in self.transactions if x is not other]
        return self
        
    def __rsub__(self, other):
        return self.__sub__(self, other)
    
    def __str__(self):
        """
        Get a string representing the block. The string contains the block id, the hash of the previous block,
        the data of the transactions and the timestamp when the block was added to the blockchain
        """
        ret = f"Block id: {self.id}."
        ret += f" " + f"Hash of the previous block: {self.hash_previous_block}\n"
        ret += f"Transactions:\n"
        # loop through transactions and add them to the string
        for trans in self.transactions:
            ret += str(trans) + "\n"
            
        ret += f"Block timestamp: {self.timestamp}\n"
        return ret
        
    @property
    def size(self):
        return len(self.transactions)
    
    @property
    def hash(self):
        return sha(str(self).encode()).hexdigest()

In [78]:
trans = Transaction("Alice", "Bob", 2, dt.now())
block = Block()
print(f"Block has {block.size} transactions")
block += trans
print(f"Block has {block.size} transactions")
print(f"Hash of the block: {block.hash}")

Block has 0 transactions
Block has 1 transactions
Hash of the block: 91cdcdbb55924ab34e7a0bd2b9a52dadb73e70d7814330922451d441a8525eeb


***

## Blockchain

El objeto *Blockchain* representará la base de datos blockchain, que guardará los distintos bloques

In [79]:
generative_block = Block()

class Blockchain(object):
    """
    Object that represents the blockchain. The blockchain will store all the block objects,
    that store the different transactions
    """
    def __init__(self):
        self.blocks = [] #store the blocks in the blockchain
        # add the generative block
        self += generative_block
        
    def __add__(self, other):
        """
        Add a block to the blockchain
        """
        if isinstance(other, Block):
            other.id = self.length
            other.timestamp = dt.now()
                      
            if self.length > 0:
                other.hash_previous_block = self.previous_block.hash
                
            self.blocks.append(other)
            
        return self
    def __radd__(self, other):
        return self.__add__(self, other)
    
    def __str__(self):
        ret = f"Blockchain with {self.length} blocks\n"
        if self.length > 0:
            ret += f"The blockchain has the following blocks:\n"
            ret += "\n"
            for block in self.blocks:
                ret += str(block)
                ret += f"Hash of the block: {block.hash}\n"
                ret += "\n"
        return ret
    
    @property
    def length(self):
        return len(self.blocks)
    
    @property
    def previous_block(self):
        return self.blocks[-1]

In [80]:
bc = Blockchain()
bc += block
print(bc)

Blockchain with 2 blocks
The blockchain has the following blocks:

Block id: 0. Hash of the previous block: 
Transactions:
Block timestamp: 2018-11-09 13:46:59.039447
Hash of the block: d1774ed00ffacfa613922f444b4344f09d6373f7008b239f483d174ff1e63ab1

Block id: 1. Hash of the previous block: d1774ed00ffacfa613922f444b4344f09d6373f7008b239f483d174ff1e63ab1
Transactions:
- Transaction of 2 coins from Alice to Bob. Transaciton timestamp: 2018-11-09 13:46:58.995939
Block timestamp: 2018-11-09 13:46:59.039447
Hash of the block: fe1c0be47a2795493ed3b54e1ed68dc5550b9f0840336b88a225a16c4cbb08a2


